<a href="https://colab.research.google.com/github/sleighton2022/datasci266-final-project/blob/main/Datasci_266_baseline_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasci 266 final project - enhancement notebook - T5 EFT + PROMPT

**Description:**

**Structure of the notebook:**

**Expected results:**





## Setup

In [1]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q datasets
!pip install -q pandas
!pip install -q rouge_score
!pip install -q nltk
!pip install bert_score
!pip install sentence-transformers
!pip install -q time
!pip install -q torchinfo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti

In [2]:
#import evaluate
import torch
my_device = "cuda:0" if torch.cuda.is_available() else "cpu"
my_device

import evaluate
import numpy as np
import time
from transformers import pipeline
from transformers import Trainer, TrainingArguments

In [3]:
#from transformers import T5Tokenizer, TFT5ForConditionalGeneration  #tensorflow implementation
from transformers import T5Tokenizer, T5ForConditionalGeneration #torch

In [4]:
'''
#!pip install git+https://github.com/sleighton2022/datasci266-final-project.git
!wget https://raw.githubusercontent.com/sleighton2022/datasci266-final-project/main/summary_utils.py
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel
'''


'\n#!pip install git+https://github.com/sleighton2022/datasci266-final-project.git\n!wget https://raw.githubusercontent.com/sleighton2022/datasci266-final-project/main/summary_utils.py\nfrom summary_utils import SummaryEvaluator, DatasetManager, SummaryModel\n'

In [5]:
'''
# Load dataset
dataset_manager = DatasetManager(dataset_name="xsum", sample_size=10)
sampled_dataset = dataset_manager.load_sampled_dataset()
'''

'\n# Load dataset\ndataset_manager = DatasetManager(dataset_name="xsum", sample_size=10)\nsampled_dataset = dataset_manager.load_sampled_dataset()\n'

In [6]:
from datasets import load_dataset
xsum_dataset = load_dataset("xsum", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [7]:
train_data = xsum_dataset['train']
train_data = train_data.shuffle(seed=42)
test_data = xsum_dataset['test']
test_data = test_data.shuffle(seed=42)
valid_data = xsum_dataset['validation']
valid_data = test_data.shuffle(seed=42)
print(len(train_data))
print(len(test_data))
print(train_data[0])


204045
11334
{'document': 'In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government\'s attachment to what they call a "com

In [8]:
#basic stats of the training set
train_min_len = np.min([len(article) for article in train_data["summary"]])
train_max_len = np.max([len(article) for article in train_data["summary"]])
train_med_len = np.median([len(article) for article in train_data["summary"]])
print(f'summary - min, max, median, {train_min_len}, {train_max_len}, {train_med_len}')


summary - min, max, median, 1, 399, 126.0


In [9]:
#subsample training and valid dataset
train_dataset = train_data.select(range(20000))
valid_dataset = valid_data.select(range(500))

#set test dataset for perf eval
test_dataset = test_data.select(range(200))

In [10]:
len(train_data)

204045

In [11]:
#global params
SUMMARY_MAX_LEN = 150
SUMMARY_MIN_LEN = 30


In [12]:
#evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

from sentence_transformers import SentenceTransformer, util
sentence_transformer = SentenceTransformer("all-mpnet-base-v2")
def calculate_vector_similarity(reference_summaries, generated_summaries):
    """
    Calculates cosine similarity between reference and generated summary embeddings.
    """
    ref_embeddings = sentence_transformer.encode(reference_summaries, convert_to_tensor=True)
    gen_embeddings = sentence_transformer.encode(generated_summaries, convert_to_tensor=True)
    cosine_scores = util.cos_sim(ref_embeddings, gen_embeddings)

    avg_similarity = cosine_scores.diagonal().mean().item()  # Average cosine similarity
    return avg_similarity

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Combined Model - the best EFT strategy + best prompt strategy


In [13]:
#TO REPLACE WITH the best "prompt" from Sean's work
PROMPT = 'summarize: '

In [14]:
model_checkpoint = 't5-base'
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

model = model.to(my_device)

embedding_layer = model.shared
max_tok_len = embedding_layer.weight.shape[1]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [15]:
#basic statistics for ariticle and summary
train_min_len = np.min([len(article) for article in train_dataset["document"]])
train_max_len = np.max([len(article) for article in train_dataset["document"]])
train_med_len = np.median([len(article) for article in train_dataset["document"]])
print(f'document - min, max, median, {train_min_len}, {train_max_len}, {train_med_len}')

train_min_len = np.min([len(article) for article in train_dataset["summary"]])
train_max_len = np.max([len(article) for article in train_dataset["summary"]])
train_med_len = np.median([len(article) for article in train_dataset["summary"]])
print(f'summary - min, max, median, {train_min_len}, {train_max_len}, {train_med_len}')

document - min, max, median, 286, 9341, 1688.0
summary - min, max, median, 34, 298, 124.5


In [16]:
# Tokenize the dataset
def tokenize_function(examples):
    inputs = [PROMPT + article for article in examples["document"]]
    model_inputs = tokenizer(inputs, max_length= max_tok_len, truncation=True, padding= "max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length= 125, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tok_train = train_dataset.map(tokenize_function, batched=True)
tok_valid = valid_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [18]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [19]:
# freeze all layers but the last layer
modules_to_freeze = [model.encoder.block[i].layer[:-1] for i in range(len(model.encoder.block)-1)]
modules_to_freeze.extend([model.decoder.block[i].layer[:] for i in range(len(model.decoder.block))])

for module in modules_to_freeze:
    for param in module.parameters():
        param.requires_grad = False  # freeze

In [20]:
print(f'number of blocks in encoder, {len(model.encoder.block)}')
print(f'number of blocks in decoder, {len(model.decoder.block)}')

number of blocks in encoder, 12
number of blocks in decoder, 12


In [21]:
total_params = sum(p.numel() for p in model.parameters())
print(f"total number of parameters in the model: {total_params}")

# count trainable parameters
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_parameters}")

total number of parameters in the model: 222903552
Number of trainable parameters: 83668224


In [22]:
10# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs= 2,
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset= tok_train,
    eval_dataset= tok_valid
)

start_time = time.time()
# train the model
trainer.train()
end_time = time.time()
train_time = end_time - start_time

metrics = trainer.evaluate()
print(metrics)

print(f'training time:  {round(train_time/60,2)} mins')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,12.072117
2,No log,11.432642


{'eval_loss': 11.432641983032227, 'eval_runtime': 3.9344, 'eval_samples_per_second': 12.708, 'eval_steps_per_second': 1.779, 'epoch': 2.0}
training time:  1.53 mins


In [23]:
#basic stats of the test set
test_dataset = test_data.select(range(200))

test_min_len = np.min([len(article) for article in test_dataset["summary"]])
test_max_len = np.max([len(article) for article in test_dataset["summary"]])
test_med_len = np.median([len(article) for article in test_dataset["summary"]])
print(f'summary - min, max, median, {test_min_len}, {test_max_len}, {test_med_len}')

summary - min, max, median, 26, 260, 125.0


generate summary on test set



In [24]:
PROMPT = 'summarize: '
t5_lora_summaries = []

start_time = time.time()
for article in test_dataset['document']:
    inputs = tokenizer(PROMPT + article, max_length= SUMMARY_MAX_LEN, truncation=True, return_tensors="pt")
    inputs = inputs.to(my_device)
    # Generate Summary
    summary_ids = model.generate(input_ids=inputs.input_ids,
                                         #max_new_tokens = max_tok_len,
                                         min_length= SUMMARY_MIN_LEN,
                                         max_length= SUMMARY_MAX_LEN,
                                         num_beams=4) #,
    candidate = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    t5_lora_summaries.append(candidate)
end_time = time.time()
inference_time = end_time - start_time

print(f'inference time:  {round(inference_time/60,2)} mins')

inference time:  3.94 mins


In [25]:
#check generated summary vs. reference
for i in range(len(test_dataset[:3])):

  print(test_dataset['summary'][i])
  print(t5_lora_summaries[i])
  print()

A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
minibus driver jailed for causing friend's death after being hit by lorry in 2013 . 21 women were heading to hen party when minibus was hit by lorry on the motorway . minibus driver was jailed for more than six years for causing friend's death .

A Tudor manor house has reopened following a £2.2m makeover.
bramall hall in stockport, greater manchester, has been restored . 1,400 tickets have been sold out for the opening weekend . the manor dates back to the reign of William the Conqueror .

Walt Disney World has unveiled a lighthouse memorial for a young boy who was killed by an alligator while on holiday at the Florida theme park.
two-year-old Lane Thomas Graves was dragged underwater by an alligator . his parents and sister had been visiting the resort from the state of Nebraska . wildlife officials classified the killing as a predatory attack .



In [26]:
#evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

from sentence_transformers import SentenceTransformer, util
sentence_transformer = SentenceTransformer("all-mpnet-base-v2")
def calculate_vector_similarity(reference_summaries, generated_summaries):
    """
    Calculates cosine similarity between reference and generated summary embeddings.
    """
    ref_embeddings = sentence_transformer.encode(reference_summaries, convert_to_tensor=True)
    gen_embeddings = sentence_transformer.encode(generated_summaries, convert_to_tensor=True)
    cosine_scores = util.cos_sim(ref_embeddings, gen_embeddings)

    avg_similarity = cosine_scores.diagonal().mean().item()  # Average cosine similarity
    return avg_similarity

In [27]:
references = test_dataset['summary']
candidates = t5_lora_summaries

rouge_outputs = rouge.compute(predictions=candidates, references=references)
print(rouge_outputs['rouge1'], rouge_outputs['rouge2'], rouge_outputs['rougeL'])
bleu_outputs = bleu.compute(predictions=candidates, references=references)
print(bleu_outputs['bleu'])
bert_outputs = bertscore.compute(predictions=candidates, references=references, lang="en")
print(np.mean(bert_outputs['f1']))
print(calculate_vector_similarity(references, candidates))
print()

0.1858736388022359 0.02289097703122575 0.12436144155424644
0.006748869453373998


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.8490046784281731
0.49853256344795227

